In [11]:
from tqdm import tqdm_notebook as tqdm
from IPython.display import Markdown, display
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle

import tensorflow as tf
from tensorflow import keras

In [12]:
SMALL_DATASET_DIR = '../../nmnist/notMNIST_small/'
LARGE_DATASET_DIR = '../../nmnist/notMNIST_large/'
CACHE = {}
LABEL_MAP = {}
INV_LABEL_MAP = {}

In [13]:
def read(data_dir, override=False):
    f_v = 0
    global CACHE
    if not CACHE.get(data_dir, []) or override:
        CACHE[data_dir] = []
        X, y = [], []
        for f in tqdm(os.listdir(data_dir), desc='Letter'):
            if not f.startswith('.'):
                img_dir = os.path.join(data_dir, f)
                for img in os.listdir(img_dir):
                    img_path = os.path.join(img_dir, img)
                    data = cv2.imread(img_path, 0)
                    if data is None:
                        continue
                    X.append(data * 2 / 255 - 1)
                    if LABEL_MAP.get(f) is None:
                        LABEL_MAP[f] = f_v
                        INV_LABEL_MAP[f_v] = f
                        f_v += 1
                    y.append(LABEL_MAP[f])
        CACHE[data_dir].append(np.array(X))
        CACHE[data_dir].append(np.array(y))
    return CACHE[data_dir][0], CACHE[data_dir][1]

In [14]:
def get_data(data_dir, verbose=False, override=False):
    X, y = read(data_dir, override=override)
    assert X.shape[0] == y.shape[0]
    N = X.shape[0]
    if verbose:
        print(X.shape)
        print(y.shape)
        print(X[:5])
        print(y[:5])
        print(np.unique(y))
    return X, y

In [15]:
def get_split_data(data_dir, size=(0.7, 0.3), verbose=False, random_state=6, override=False):
    X, y = shuffle(*get_data(data_dir, verbose=verbose, override=override), random_state=random_state)
    assert abs(np.sum(size) - 1.0) < 0.001
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size[1], random_state=random_state)
    return X_train, y_train, X_test, y_test

In [16]:
X_train, y_train, X_test, y_test = get_split_data(SMALL_DATASET_DIR, override=False, verbose=False)
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

$$28x28 \rightarrow (conv: f=3, s=1, c=6) \rightarrow 26x26x6 \rightarrow (conv: f=4, s=2, c=8) \rightarrow 11x11x8 \rightarrow flatten \rightarrow 968 \rightarrow FC \rightarrow 120 \rightarrow softmax$$

In [17]:
lr = 0.1
epochs = 20
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.Conv2D(8, kernel_size=(4, 4), strides=(2, 2), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.1
Epochs: 20

Epoch 1/20
13106/13106 [==============================] - 7s 500us/step - loss: 0.8174 - acc: 0.7582
Epoch 2/20
13106/13106 [==============================] - 5s 387us/step - loss: 0.4163 - acc: 0.8765
Epoch 3/20
13106/13106 [==============================] - 5s 403us/step - loss: 0.3321 - acc: 0.8997
Epoch 4/20
13106/13106 [==============================] - 6s 420us/step - loss: 0.3053 - acc: 0.9096
Epoch 5/20
13106/13106 [==============================] - 6s 427us/step - loss: 0.2204 - acc: 0.9332
Epoch 6/20
13106/13106 [==============================] - 7s 532us/step - loss: 0.1806 - acc: 0.9418
Epoch 7/20
13106/13106 [==============================] - 5s 414us/step - loss: 0.1347 - acc: 0.9587
Epoch 8/20
13106/13106 [==============================] - 6s 465us/step - loss: 0.1085 - acc: 0.9669
Epoch 9/20
13106/13106 [==============================] - 5s 383us/step - loss: 0.0883 - acc: 0.9729
Epoch 10/20
13106/13106 [===============

Vertical:
$$28x28 \rightarrow (conv: f=(5, 3), s=1, c=6) \rightarrow 24x26x6 \rightarrow (conv: f=4, s=2, c=8) \rightarrow 11x12x8 \rightarrow flatten \rightarrow 1056 \rightarrow FC \rightarrow 256 \rightarrow FC \rightarrow 64 \rightarrow softmax$$

In [38]:
lr = 0.1
epochs = 20
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(5, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.Conv2D(8, kernel_size=(4, 4), strides=(2, 2), activation='relu'),
    keras.layers.Flatten(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.1
Epochs: 20

Epoch 1/20
13106/13106 [==============================] - 7s 503us/step - loss: 0.9692 - acc: 0.6837
Epoch 2/20
13106/13106 [==============================] - 6s 439us/step - loss: 0.3179 - acc: 0.9043
Epoch 3/20
13106/13106 [==============================] - 5s 414us/step - loss: 0.2303 - acc: 0.9277
Epoch 4/20
13106/13106 [==============================] - 6s 449us/step - loss: 0.1774 - acc: 0.9470
Epoch 5/20
13106/13106 [==============================] - 7s 527us/step - loss: 0.1221 - acc: 0.9610
Epoch 6/20
13106/13106 [==============================] - 5s 392us/step - loss: 0.0849 - acc: 0.9730
Epoch 7/20
13106/13106 [==============================] - 5s 390us/step - loss: 0.1013 - acc: 0.9717
Epoch 8/20
13106/13106 [==============================] - 5s 405us/step - loss: 0.0736 - acc: 0.9770
Epoch 9/20
13106/13106 [==============================] - 5s 383us/step - loss: 0.0571 - acc: 0.9826
Epoch 10/20
13106/13106 [===============

LeNet-5 + vertical:
$$
28x28 \rightarrow (conv: f=(5, 3), s=1, c=6, tanh) \rightarrow 24x26x6 \rightarrow (maxpool: f=2, s=2) \rightarrow 12x13x6 \rightarrow (conv: f=3, s=1, c=8, tanh) \rightarrow 10x11x8 \rightarrow (maxpool: f=2, s=2) \rightarrow 5x5x8 \rightarrow flatten \rightarrow 200 \rightarrow FC(relu) \rightarrow 120 \rightarrow FC(relu) \rightarrow 84 \rightarrow softmax
$$

In [40]:
lr = 0.1
epochs = 20
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(5, 3), strides=(1, 1), activation='tanh', input_shape=(28, 28, 1)),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='tanh'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(84, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.1
Epochs: 20

Epoch 1/20
13106/13106 [==============================] - 8s 605us/step - loss: 0.7078 - acc: 0.7890
Epoch 2/20
13106/13106 [==============================] - 9s 703us/step - loss: 0.3711 - acc: 0.8865
Epoch 3/20
13106/13106 [==============================] - 7s 540us/step - loss: 0.3057 - acc: 0.9054
Epoch 4/20
13106/13106 [==============================] - 8s 603us/step - loss: 0.2578 - acc: 0.9201
Epoch 5/20
13106/13106 [==============================] - 7s 533us/step - loss: 0.2250 - acc: 0.9289
Epoch 6/20
13106/13106 [==============================] - 7s 530us/step - loss: 0.1926 - acc: 0.9382
Epoch 7/20
13106/13106 [==============================] - 7s 537us/step - loss: 0.1675 - acc: 0.9469
Epoch 8/20
13106/13106 [==============================] - 8s 577us/step - loss: 0.1428 - acc: 0.9554
Epoch 9/20
13106/13106 [==============================] - 7s 522us/step - loss: 0.1246 - acc: 0.9626
Epoch 10/20
13106/13106 [===============

LeNet-5 + vertical:
$$
28x28 \rightarrow (conv: f=(5, 3), s=1, c=6, tanh) \rightarrow 24x26x6 \rightarrow (maxpool: f=2, s=2) \rightarrow 12x13x6 \rightarrow (conv: f=3, s=1, c=8, tanh) \rightarrow 10x11x8 \rightarrow (maxpool: f=2, s=2) \rightarrow 5x5x8 \rightarrow flatten \rightarrow 200 \rightarrow FC(tanh) \rightarrow 120 \rightarrow FC(tanh) \rightarrow 84 \rightarrow softmax
$$

In [41]:
lr = 0.1
epochs = 20
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(5, 3), strides=(1, 1), activation='tanh', input_shape=(28, 28, 1)),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(8, kernel_size=(3, 3), strides=(1, 1), activation='tanh'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='tanh'),
    keras.layers.Dense(84, activation='tanh'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.1
Epochs: 20

Epoch 1/20
13106/13106 [==============================] - 9s 660us/step - loss: 0.6543 - acc: 0.8101
Epoch 2/20
13106/13106 [==============================] - 7s 546us/step - loss: 0.3689 - acc: 0.8917
Epoch 3/20
13106/13106 [==============================] - 7s 570us/step - loss: 0.3051 - acc: 0.9099
Epoch 4/20
13106/13106 [==============================] - 8s 600us/step - loss: 0.2604 - acc: 0.9247
Epoch 5/20
13106/13106 [==============================] - 7s 545us/step - loss: 0.2284 - acc: 0.9321
Epoch 6/20
13106/13106 [==============================] - 7s 540us/step - loss: 0.2011 - acc: 0.9390
Epoch 7/20
13106/13106 [==============================] - 7s 538us/step - loss: 0.1770 - acc: 0.9480
Epoch 8/20
13106/13106 [==============================] - 7s 536us/step - loss: 0.1554 - acc: 0.9538
Epoch 9/20
13106/13106 [==============================] - 7s 564us/step - loss: 0.1359 - acc: 0.9593
Epoch 10/20
13106/13106 [===============

Custom with maxpool:
$$28x28 \rightarrow (conv: f=3, s=1, c=6) \rightarrow 26x26x6 \rightarrow (maxpool: f=2, s=2) \rightarrow 13x13x6 \rightarrow (conv: f=3, s=2, c=16) \rightarrow 5x5x16 \rightarrow (maxpool: f=2, s=1) \rightarrow 4x4x16 \rightarrow flatten \rightarrow 258 \rightarrow FC \rightarrow 120 \rightarrow softmax$$

In [35]:
lr = 0.1
epochs = 20
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(16, kernel_size=(3, 3), strides=(2, 2), activation='relu'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.1
Epochs: 20

Epoch 1/20
13106/13106 [==============================] - 6s 460us/step - loss: 0.7578 - acc: 0.7618
Epoch 2/20
13106/13106 [==============================] - 6s 480us/step - loss: 0.3381 - acc: 0.8976
Epoch 3/20
13106/13106 [==============================] - 5s 393us/step - loss: 0.2763 - acc: 0.9141
Epoch 4/20
13106/13106 [==============================] - 6s 462us/step - loss: 0.2359 - acc: 0.9284
Epoch 5/20
13106/13106 [==============================] - 7s 510us/step - loss: 0.2048 - acc: 0.9354
Epoch 6/20
13106/13106 [==============================] - 5s 416us/step - loss: 0.1842 - acc: 0.9416
Epoch 7/20
13106/13106 [==============================] - 6s 472us/step - loss: 0.1631 - acc: 0.9495
Epoch 8/20
13106/13106 [==============================] - 6s 431us/step - loss: 0.1390 - acc: 0.9549
Epoch 9/20
13106/13106 [==============================] - 5s 409us/step - loss: 0.1337 - acc: 0.9561
Epoch 10/20
13106/13106 [===============

Custom with avgpool:
$$28x28 \rightarrow (conv: f=3, s=1, c=6) \rightarrow 26x26x6 \rightarrow (avgpool: f=2, s=2) \rightarrow 13x13x6 \rightarrow (conv: f=3, s=2, c=16) \rightarrow 5x5x16 \rightarrow (avgpool: f=2, s=1) \rightarrow 4x4x16 \rightarrow flatten \rightarrow 258 \rightarrow FC \rightarrow 120 \rightarrow softmax$$

In [36]:
lr = 0.1
epochs = 20
print('Building new net...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(3, 3), strides=(1, 1), activation='relu', input_shape=(28, 28, 1)),
    keras.layers.AvgPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(16, kernel_size=(3, 3), strides=(2, 2), activation='relu'),
    keras.layers.AvgPool2D(pool_size=(2, 2), strides=(1, 1)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='relu'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building new net...
Learning rate: 0.1
Epochs: 20

Epoch 1/20
13106/13106 [==============================] - 7s 554us/step - loss: 0.8933 - acc: 0.7232
Epoch 2/20
13106/13106 [==============================] - 5s 402us/step - loss: 0.3906 - acc: 0.8840
Epoch 3/20
13106/13106 [==============================] - 5s 372us/step - loss: 0.3174 - acc: 0.9039
Epoch 4/20
13106/13106 [==============================] - 6s 488us/step - loss: 0.2792 - acc: 0.9129
Epoch 5/20
13106/13106 [==============================] - 5s 394us/step - loss: 0.2504 - acc: 0.9235
Epoch 6/20
13106/13106 [==============================] - 4s 334us/step - loss: 0.2276 - acc: 0.9286
Epoch 7/20
13106/13106 [==============================] - 4s 336us/step - loss: 0.2128 - acc: 0.9315
Epoch 8/20
13106/13106 [==============================] - 4s 341us/step - loss: 0.1951 - acc: 0.9393
Epoch 9/20
13106/13106 [==============================] - 5s 366us/step - loss: 0.1827 - acc: 0.9416
Epoch 10/20
13106/13106 [===============

LeNet-5:
$$
28x28 \rightarrow (conv: f=5, s=1, c=6, tanh) \rightarrow 24x24x6 \rightarrow (maxpool: f=2, s=2) \rightarrow 12x12x6 \rightarrow (conv: f=5, s=1, c=16) \rightarrow 8x8x16 \rightarrow (maxpool: f=2, s=2) \rightarrow 4x4x16 \rightarrow flatten \rightarrow 256 \rightarrow FC \rightarrow 128 \rightarrow FC \rightarrow 64 \rightarrow softmax
$$

In [37]:
lr = 0.1
epochs = 20
print('Building LeNet-5...')
print(f'Learning rate: {lr}')
print(f'Epochs: {epochs}')
print()
model = keras.Sequential([
    keras.layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=(28, 28, 1)),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh'),
    keras.layers.MaxPool2D(pool_size=(2, 2), strides=(2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(120, activation='tanh'),
    keras.layers.Dense(84, activation='tanh'),
    keras.layers.Dense(10, activation=tf.nn.softmax)
])
model.compile(optimizer=tf.train.GradientDescentOptimizer(lr), 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train, y_train, epochs=epochs)
loss, acc = model.evaluate(X_test, y_test)
print(f'Loss: {loss}')
print(f'Accuracy: {acc}')
print('=======================')

Building LeNet-5...
Learning rate: 0.1
Epochs: 20

Epoch 1/20
13106/13106 [==============================] - 9s 703us/step - loss: 0.6139 - acc: 0.8253
Epoch 2/20
13106/13106 [==============================] - 9s 716us/step - loss: 0.3443 - acc: 0.8984
Epoch 3/20
13106/13106 [==============================] - 9s 660us/step - loss: 0.2777 - acc: 0.9181
Epoch 4/20
13106/13106 [==============================] - 9s 650us/step - loss: 0.2333 - acc: 0.9300
Epoch 5/20
13106/13106 [==============================] - 9s 656us/step - loss: 0.1996 - acc: 0.9397
Epoch 6/20
13106/13106 [==============================] - 8s 590us/step - loss: 0.1737 - acc: 0.9469
Epoch 7/20
13106/13106 [==============================] - 8s 587us/step - loss: 0.1501 - acc: 0.9555
Epoch 8/20
13106/13106 [==============================] - 8s 595us/step - loss: 0.1299 - acc: 0.9602
Epoch 9/20
13106/13106 [==============================] - 8s 612us/step - loss: 0.1068 - acc: 0.9696
Epoch 10/20
13106/13106 [===============